# Functions

In [ ]:
library(cisTopic)

library(Rtsne)

library(umap)

run_umap <- function(fm_mat){
    umap_object = umap(t(fm_mat),random_state = 2019)
    df_umap = umap_object$layout
    return(df_umap)
}


fun_all <- function(regions){
    suppressMessages({
        cisTopicObject <- createcisTopicObjectFromBAM(bamFiles, regions, project.name='buenrostro2018',paired = TRUE)
        cisTopicObject <- renameCells(cisTopicObject, cellnames)
        cisTopicObject <- runCGSModels(cisTopicObject, topic=c(10, 20, 25, 30, 35, 40), seed=987, nCores=10, burnin = 120, iterations = 150, addModels=FALSE)
        cisTopicObject <- selectModel(cisTopicObject)
    })


    cellassign <- modelMatSelection(cisTopicObject, 'cell', 'Probability')

    df_out <- cellassign

    tsne <- Rtsne(t(df_out), perplexity = 50, 
                           check_duplicates = FALSE, pca=FALSE, theta=0.01, max_iter=1000)$Y


    umap <- run_umap(df_out[!is.na(rowSums(df_out)),])

    res <- list(df_out=df_out, umap=umap, tsne=tsne)
}


### Obtain Feature Matrix

In [ ]:
start_time = Sys.time()

In [ ]:
set.seed(2019)

In [ ]:
metadata <- read.table('../../raw_data/input/metadata.tsv',
                         header = TRUE,
                         stringsAsFactors=FALSE,quote="",row.names=1)

In [ ]:
pathToBams <- '../../raw_data/bam/files/sc-bams_nodup/'

In [ ]:
bamFiles <- paste(pathToBams, list.files(pathToBams), sep='')

In [ ]:
cellnames <- sapply(strsplit(basename(bamFiles),'.',fixed = TRUE), "[[", 1)
head(cellnames)

In [ ]:
ix = match(rownames(metadata),cellnames)
bamFiles = bamFiles[ix]
cellnames = cellnames[ix]

In [ ]:
regions <- '../../cpeaks_filteredFeature/combinedPeaks.top.regions.bed'

In [ ]:
suppressMessages({
    cisTopicObject <- createcisTopicObjectFromBAM(bamFiles, regions, project.name='buenrostro2018',paired = TRUE)
    cisTopicObject <- renameCells(cisTopicObject, cellnames)
    
    cisTopicObject <- runCGSModels(cisTopicObject, topic=c(10, 20, 25, 30, 35, 40), seed=987, nCores=10, burnin = 120, iterations = 150, addModels=FALSE)

    cisTopicObject <- selectModel(cisTopicObject)
    
    df_out <- cellassign

    tsne <- Rtsne(t(df_out), perplexity = 50, 
                           check_duplicates = FALSE, pca=FALSE, theta=0.01, max_iter=1000)

    umap <- run_umap(df_out[!is.na(rowSums(res.top$df_out)),])

    res.all <- list(df_out=df_out, umap=umap, tsne=tsne)
})


In [ ]:
df_out <- cellassign

tsne <- Rtsne(t(df_out), perplexity = 50, 
                       check_duplicates = FALSE, pca=FALSE, theta=0.01, max_iter=1000)$Y


umap <- run_umap(df_out[!is.na(rowSums(df_out)),])

res.top <- list(df_out=df_out, umap=umap, tsne=tsne)

In [ ]:
regions <- '../../datafr/regions/bulk.top2.regions.bed'

res.top2 <- fun_all(regions)

In [ ]:
regions <- '../../../../../test_data/Buenrostro_2018/input/GSE96769_PeakFile_20160207.bed'

In [ ]:
res.all <- fun_all(regions)

In [ ]:
library('tidyverse')

In [ ]:
rds.list <- list(res.all=res.all,
    res.top=res.top,
                 res.top2=res.top2
                 
    # res.hk=res.hk,
    # res.hk.excl=res.hk.excl,
    # res.b1=res.b1,
    # res.b2=res.b2
    # res.hk.long.excl=res.hk.long.excl,
    # res.hk.long.lowrate.excl
                )

rds.list%>%names%>%lapply(function(x){
    
    saveRDS(rds.list[[x]]$df_out,file = paste0('./rds/FM_bulkPeaks-',gsub('\\.','-',x),'_data1.rds'))
    
})